# Exempel notebook

1. Läsa in data från SAS dataset
* Hur man skapar en ny variabel baserat på befintlig variabel
* Hur man konverterar ett string objekt till Pandas datetime variabel
* Några enkla metoder för att förstå sitt data
* Gruppering av data
* Filtrering - välja rader på kriterium
* Flödeslogik och villkor i en funktion applicerad mot dataframe
* Joina dataframes
* Konkatinera dataframes
* Visualisering - Matplotlib

In [2]:
import pandas as pd
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt

In [3]:
?pd.read_sas

In [4]:
type(pd.read_sas)

function

## 1. Läsa in data från SAS format

In [5]:
df_staff = pd.read_sas('staff.sas7bdat', encoding = 'Latin8')

In [6]:
# Information om det inlästa datat
df_staff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 10 columns):
Employee_ID      424 non-null float64
Job_Title        424 non-null object
Salary           424 non-null float64
Gender           424 non-null object
Birth_Date       424 non-null float64
Emp_Hire_Date    424 non-null float64
Emp_Term_Date    116 non-null float64
Manager_ID       423 non-null float64
start_date       424 non-null object
end_date         424 non-null object
dtypes: float64(6), object(4)
memory usage: 33.2+ KB


In [7]:
# Tittar på de 5 första observationerna
df_staff.head()

,Employee_ID,Job_Title,Salary,Gender,Birth_Date,Emp_Hire_Date,Emp_Term_Date,Manager_ID,start_date,end_date
0,120101.0,Director,163040.0,M,7535.0,17348.0,NaN,120261.0,2007-07-01,9999-12-31
1,120102.0,Sales Manager,108255.0,M,4971.0,12205.0,NaN,120101.0,1993-06-01,9999-12-31
2,120103.0,Sales Manager,87975.0,M,-2535.0,6575.0,NaN,120101.0,1978-01-01,9999-12-31
3,120104.0,Administration Manager,46230.0,F,-600.0,9132.0,NaN,120101.0,1985-01-01,9999-12-31
4,120105.0,Secretary I,27110.0,F,6929.0,15826.0,NaN,120101.0,2003-05-01,9999-12-31


### Dataframen innehåller två datatyper: float64 samt object
* float64 - numerisk variabel med decimaler
* object - pandas datatyp för character

## 2. Hur skapar man en ny variabel i Pandas

In [ ]:
df_staff['Double'] = df_staff['Salary'] * 2

In [ ]:
df_staff.head()

## 3. Konvertera till datetimeformat

### Vi behöver konvertera datumvariablerna (Start_Date samt End:Date) till datetime - datumvariabel i Pandas. Nu ligger den som objekt och då kan vi inte applicera datum metoder på denna. För att göra den konverteringen använder vi pandas funktion to_datetime


In [8]:
?pd.to_datetime

In [9]:
df_staff['P_start_date'] = pd.to_datetime(df_staff['start_date'])

In [10]:
df_staff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 11 columns):
Employee_ID      424 non-null float64
Job_Title        424 non-null object
Salary           424 non-null float64
Gender           424 non-null object
Birth_Date       424 non-null float64
Emp_Hire_Date    424 non-null float64
Emp_Term_Date    116 non-null float64
Manager_ID       423 non-null float64
start_date       424 non-null object
end_date         424 non-null object
P_start_date     424 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(6), object(4)
memory usage: 36.5+ KB


In [ ]:
df_staff.head()

### Nu kan vi applicera metoder mot datumvariabeln

In [11]:
type(df_staff['P_start_date'])

pandas.core.series.Series

In [12]:
# För att se vilka attribut som finns tillgängliga på en serie
test = df_staff['P_start_date']

In [13]:
dir(test)

['T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_SLICEMAP',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_prepare__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__bytes__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 

In [ ]:
?test.dt

### Vi skapar år, månad och dag

In [ ]:
df_staff['Year'] = df_staff['P_start_date'].dt.year
df_staff['Month'] = df_staff['P_start_date'].dt.month
df_staff['day'] = df_staff['P_start_date'].dt.month


In [ ]:
df_staff.head()

## 4. Exempel på metoder för att enkelt förstå sitt data

In [ ]:
# Fördelning män och kvinnor
df_staff['Gender'].value_counts()

In [ ]:
# Medelvärde lön
df_staff['Salary'].mean()


In [ ]:
# Univariat statistik på alla numeriska variabler i dataframen
df_staff.describe()

## 5. Gruppering av data. 
### För att gruppera sitt data skapar man ett sk grouped objekt. Ett grouped objekt innehåller metadata som beskriver hur grupperingen är gjord. På detta objekt kan man sedan välja kolumner att ta fram olika statistik på. Detta sker - det vet ni nu - självklart med en lämplig metod.

In [ ]:
# Vi skapar ett grouped objekt på Gender

grouped = df_staff.groupby('Gender')

In [ ]:
type(grouped)

In [ ]:
# Finns det någon skillnad i lön mellan män och kvinnor?. Väljer Serien Salery från groupedby objektet och applicerar 
# metoden mean

grouped['Salary'].mean()

In [ ]:
### Man kan självklart gruppera på mer än en variabel. Observera att man då lägger grupperingsvariablerna i en lista

grouped_multi =  df_staff.groupby(['Year','Gender'])

In [ ]:
round(grouped_multi['Salary'].mean())

## 6. Filtrering - välja rader på kriterium

In [14]:
# Vi skapar en ny dataframe som enbart innehåller kvinnor
df_staff_female = df_staff[df_staff['Gender'] == 'F']

In [15]:
df_staff_female.count()

Employee_ID      191
Job_Title        191
Salary           191
Gender           191
Birth_Date       191
Emp_Hire_Date    191
Emp_Term_Date     52
Manager_ID       191
start_date       191
end_date         191
P_start_date     191
dtype: int64

### Vad är det vi gör här egentligen?

In [16]:
df_staff['Truth'] = df_staff['Gender'] == 'F'

In [17]:
# Väljer ett subset av variabler
df_staff[['Gender','Truth']].head()

,Gender,Truth
0,M,False
1,M,False
2,M,False
3,F,True
4,F,True


In [ ]:
### Subsetting with Calculated Values - exempel SQL

'''
proc sql;
select Gender, Salary, Bonus,
       Salary * .10 as Bonus
   from orion.employee_payroll
   where calculated Bonus < 3000;
quit; 

'''


In [18]:
df_staff['Bonus'] = df_staff['Salary'] * 0.1
df_staff_subset = df_staff[df_staff['Bonus'] > 3000][[ 'Gender', 'Salary','Bonus']]

In [19]:
df_staff_subset.head(10)

,Gender,Salary,Bonus
0,M,163040.0,16304.0
1,M,108255.0,10825.5
2,M,87975.0,8797.5
3,F,46230.0,4623.0
6,F,30475.0,3047.5
13,F,31285.0,3128.5
16,M,31670.0,3167.0
18,M,30255.0,3025.5
24,M,32040.0,3204.0
27,F,30890.0,3089.0


In [ ]:
df_staff_subset['Gender'].value_counts()

## 7. Flödeslogik och villkor i en funktion applicerad mot dataframe
### Förfina logiken med egendiefinerad funktion som ancänds av apply metoden i en lambda funktion. Om det är en kvinna ska bonusen vara 20%, om en man -20%.

In [20]:
def metoo(gender, salery):
    if gender == 'F':
        bonus = salery * 0.2
    else:
        bonus = salery * (-0.2)
    return bonus

In [21]:
bonus = metoo('M',100)
bonus

-20.0

### OBS! Förstår man nedan har man ett mycket kraftfullt verktyg för att manipupelera data på radnivå med Pandas

In [ ]:
df_staff['Mee_to_Bonus'] = df_staff.apply(lambda x: metoo(x['Gender'], x['Salary']), axis = 1) 

### Metoden apply appliceras på objektet df_staff. Denna metod tillåter att man skickar in en funktion. I exemplet ovan är det en lambda funktion som använder en egendefineradfunktion som argument (metoo). På varje rad appliceras funktionen metoo med argumenten df_staff['Gender'] och df_staff['Salary]. Lambda funktionen erbjuder möjligheter att använda funktioner i sammanhang där den vanliga funktionen inte fungerar. Det är väl använd tid att sätta sig in i lambda konceptet.

In [ ]:
df_staff[['Gender','Salary','Mee_to_Bonus']].head(10)

In [ ]:
?df_staff.apply

In [ ]:
type(df_staff.apply)

## 8. Joina dataframes 

In [ ]:
df_cust = pd.read_sas('customer2.sas7bdat', encoding = 'Latin8')
df_trans = pd.read_sas('transaction2.sas7bdat', encoding = 'Latin8')

In [ ]:
df_cust.info()

In [ ]:
df_trans.info()

In [ ]:
df_cust.head()

In [ ]:
df_cust = df_cust.drop_duplicates('ID')
df_cust.head()

In [ ]:
df_trans.head()

In [ ]:
# Inner join
df_merged_inner = df_cust.merge(df_trans, on = ['ID'], how = 'inner')
df_merged_inner.head()


In [ ]:
# Left join 

df_merged_left = df_cust.merge(df_trans, on = ['ID'], how = 'left')
df_merged_left.head()

In [ ]:
# Outer join

df_merged_outer = df_cust.merge(df_trans, on = ['ID'], how = 'outer')
df_merged_outer.head(6)

## 9. Konkatinera dataframes

In [ ]:
df_merged = df_merged_outer.copy() 

In [ ]:
df_merged

In [ ]:
concat_df = pd.concat([df_merged, df_merged_outer ], axis = 0)

In [ ]:
concat_df.head(20)

## 9. Split - Apply - Combine
### Aggregering - summera data i en serie för att returnera en skalär.  Vi vill ta fram statistik över en gruppering på data: split- apply-combine konceptet

* Split - Datat delas upp i delar på vald grupperingsvariabel
* Apply - På delarna appliceras logik som returnerar skalär
* Combine - Delarna sätts ihop på grupperingsvariabeln

In [ ]:
# Läser in nytt exempeldata.

df_order_fact = pd.read_sas('order_fact.sas7bdat', encoding = 'Latin8')
df_product_dim = pd.read_sas('product_dim.sas7bdat', encoding = 'Latin8')

In [ ]:
df_product_dim.info()

In [ ]:
df_product_dim.head()

In [ ]:
df_order_fact.info()

In [ ]:
### Joinar fakta tabell mot dimensonstabellen och lägger på produktkategori

df_analys = df_order_fact.merge(df_product_dim[['Product_ID','Product_Line','Product_Category']],
                                on = ['Product_ID'], how = 'inner')

In [ ]:
df_analys.info()

In [ ]:
df_analys['Product_Category'].value_counts()

In [ ]:
df_analys['Product_Line'].value_counts()

In [ ]:
# Andel av totalen
round(df_analys['Product_Line'].value_counts()/len(df_analys),2)

### Nu vill vi ta fram statistik på grupperingen Product Line - vi använder split - apply - combine konceptet. Först med Pandas optimerade metoder. Dessa anropas med agg metoden - returnerar skalär 

In [ ]:
grouped = df_analys.groupby(['Product_Line','Product_Category']) 

In [ ]:
result = grouped['Total_Retail_Price'].agg('mean')
result

In [ ]:
### Du kan använda flera funktioner i samma anrop
funtions = ['count','mean','max']
result1 = grouped['Total_Retail_Price'].agg(funtions)
result1



### Om vi inte är nöjda med de defaulta namnen -använd en tuple

In [ ]:
### Du kan använda flera funktioner i samma anrop
funtions2 = [('N','count'),('Medel','mean'),('Max','max')]

result2 = grouped['Total_Retail_Price'].agg(funtions2)
result2

### Slutligen gör vi en egen funktion som vi använder apply mot

### Vi vill plocka de fem högs betalda anställda per kön i df_staff

In [ ]:
df_staff.info()

In [ ]:
# Gör din egna funktion
def top(df, n = 5, column ='Salary'): 
    return df.sort_values(by = column)[-n:]

In [ ]:
top(df_staff)

In [ ]:
# Nu grupperar vi på kön och plockar de med högsta lönerna per grupp

df_staff.groupby('Gender').apply(top)

## 10. Visualisering - Matplotlib
### Det finns ett helt ekosystem för visaulisering. Grundmodulen för detta är matplotlib som redan är importerad i denna notebook. Vi ska göra två enkla visualiseringar för att visa konceptet

In [ ]:
# Definierar hela arean
fig = plt.figure(figsize = (10,5))
# Skapar grafobjekt som ska läggas in i arean ovan
ax = fig.add_subplot(1,1,1)

ax.set_title('Inkomst fördelat på kön')

bar_serie = round(df_staff.groupby('Gender')['Salary'].mean())

bar_serie.plot(kind='bar', rot = 0, grid = False, alpha = 0.6)

plt.show()

### Vad ska man tänka på här? Det finns oändligt många parametrar att sätta?
### Alla dataframes och serier har en plot metod. 
### Om man sätter ett index blir detta index alltid X axeln
### Det enda sättet är att pröva sig fram med olika grafer, se hjälpen nedan
### Bra att känna till är att man kan annotera samt skapa flera grafer i samma area


In [ ]:
?bar_serie.plot

### Exenmpel annotering samt två grafer i samma bild

In [ ]:
fig = plt.figure(figsize = (18,9))
# Skapar grafobjekt som ska läggas in i arean ovan
# Graf1
ax1 = fig.add_subplot(2,1,1)
# Graf2
ax2 = fig.add_subplot(2,1,2)

ax1.set_title('Inkomst fördelat på kön')

ax2.set_title('Antal anställda per år')

bar_serie = round(df_staff.groupby('Gender')['Salary'].mean())

line_serie = round(df_staff.groupby('Year')['Gender'].count())

bar_serie.plot(kind='bar', ax = ax1, rot = 0, grid = False, alpha = 0.6)

line_serie.plot(kind='line', ax = ax2, rot = 0, style = 'k-')

### Nedan logik för annotering i graf2

ax2.annotate('Vad händer 2010?', 
            xy = (2010, 95),
            xytext = (2000, 80),
            arrowprops = dict(facecolor = 'black', shrink = 0.1, width = 2),
            horizontalalignment = 'left')

plt.show()

In [ ]:
?ax2.annotate